In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re

import warnings
warnings.filterwarnings('ignore')

# Data Cleaning 

In [41]:
#opening the data
data= pd.read_csv('data/food-inspections.csv',delimiter=',')

display(data.head(3))

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,...,Results,Violations,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,2320315,SERENDIPITY CHILDCARE,SERENDIPITY CHILDCARE,2216009.0,Daycare Above and Under 2 Years,Risk 1 (High),1300 W 99TH ST,CHICAGO,IL,60643.0,...,Pass,NaN,41.714168,-87.655291,"{'longitude': '41.7141680989703', 'latitude': ...",NaN,NaN,NaN,NaN,NaN
1,2320342,YOLK TEST KITCHEN,YOLK TEST KITCHEN,2589655.0,Restaurant,Risk 1 (High),1767 N MILWAUKEE AVE,CHICAGO,IL,60647.0,...,Pass w/ Conditions,23. PROPER DATE MARKING AND DISPOSITION - Comm...,41.913588,-87.682203,"{'longitude': '41.9135877900482', 'latitude': ...",NaN,NaN,NaN,NaN,NaN
2,2320328,LAS ASADAS MEXICAN GRILL,LAS ASADAS MEXICAN GRILL,2583309.0,Restaurant,Risk 1 (High),3834 W 47TH ST,CHICAGO,IL,60632.0,...,Out of Business,NaN,41.808025,-87.720037,"{'longitude': '41.80802515275297', 'latitude':...",NaN,NaN,NaN,NaN,NaN


### 1- General cleaning

*In this first part, we will do general cleaning : homogenize the format of our dataframe and remove the duplicates, empty columns, NA values and null values.*

In [42]:
# drop all the empty columns
data.drop(['Historical Wards 2003-2015', 'Zip Codes', 'Community Areas','Census Tracts','Wards'], axis=1,inplace=True)

#show the dataframe
display(data.head(3))

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2320315,SERENDIPITY CHILDCARE,SERENDIPITY CHILDCARE,2216009.0,Daycare Above and Under 2 Years,Risk 1 (High),1300 W 99TH ST,CHICAGO,IL,60643.0,2019-10-23T00:00:00.000,License Re-Inspection,Pass,NaN,41.714168,-87.655291,"{'longitude': '41.7141680989703', 'latitude': ..."
1,2320342,YOLK TEST KITCHEN,YOLK TEST KITCHEN,2589655.0,Restaurant,Risk 1 (High),1767 N MILWAUKEE AVE,CHICAGO,IL,60647.0,2019-10-23T00:00:00.000,Canvass,Pass w/ Conditions,23. PROPER DATE MARKING AND DISPOSITION - Comm...,41.913588,-87.682203,"{'longitude': '41.9135877900482', 'latitude': ..."
2,2320328,LAS ASADAS MEXICAN GRILL,LAS ASADAS MEXICAN GRILL,2583309.0,Restaurant,Risk 1 (High),3834 W 47TH ST,CHICAGO,IL,60632.0,2019-10-23T00:00:00.000,Canvass,Out of Business,NaN,41.808025,-87.720037,"{'longitude': '41.80802515275297', 'latitude':..."


> We can create a function that standardize columns format in order to make the data more friendly to use. It fills empty space by '_' and lower case all the letters .

In [43]:
def standardize(column):
    
    column=column.replace("#", "")
    column = column.lower().replace(" ", "_")
    
    if len(column) > 1:
        if column[-1] == "_":
            return column[:-1]
    return column

#application of the function to the dataset
data.columns = [standardize(x) for x in data.columns]

display(data.columns)


Index(['inspection_id', 'dba_name', 'aka_name', 'license', 'facility_type',
       'risk', 'address', 'city', 'state', 'zip', 'inspection_date',
       'inspection_type', 'results', 'violations', 'latitude', 'longitude',
       'location'],
      dtype='object')

> We need to check if the inspection ID is unique. If it is not the case ,we need to remove the duplicates as an ID reffers to an unique inspection.

In [44]:
#Check is the inspection ID is unique
display(data['inspection_id'].is_unique)

False

In [45]:
#Removing the duplicates 
data.drop_duplicates('inspection_id', inplace=True)

#check if all the duplicates were removed 
display(data['inspection_id'].is_unique)

True

> We need to remove all the NA values since we can't use this kind of information. But we need to remove them only from particular column ( as an example, the NA is the  violations columns only means that there were no violations so we need to keep it).

In [46]:
#Remove NA values from relevent columns
data.dropna(subset=['inspection_date','license','latitude','longitude','inspection_type'],inplace=True)

> If we explore the license numbers, we find that there are some null license number. We need to remove them.

In [47]:
# Drop "0.0" licenses
data = data[data.license != 0.0]

### 2- Cleaning the column *inspection_date*

*We need to remove the time of inspection (useless information) in order to clean the inspection date column.*

In [48]:
#We remove the time ( all the caracters after 'T')
data['inspection_date']=data['inspection_date'].apply(lambda x : x.split('T')[0])

#Converting inspection dates into float numbers
data.inspection_date = data.inspection_date.astype('datetime64[ns]')

display(data.head(3))

,inspection_id,dba_name,aka_name,license,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,2320315,SERENDIPITY CHILDCARE,SERENDIPITY CHILDCARE,2216009.0,Daycare Above and Under 2 Years,Risk 1 (High),1300 W 99TH ST,CHICAGO,IL,60643.0,2019-10-23,License Re-Inspection,Pass,NaN,41.714168,-87.655291,"{'longitude': '41.7141680989703', 'latitude': ..."
1,2320342,YOLK TEST KITCHEN,YOLK TEST KITCHEN,2589655.0,Restaurant,Risk 1 (High),1767 N MILWAUKEE AVE,CHICAGO,IL,60647.0,2019-10-23,Canvass,Pass w/ Conditions,23. PROPER DATE MARKING AND DISPOSITION - Comm...,41.913588,-87.682203,"{'longitude': '41.9135877900482', 'latitude': ..."
2,2320328,LAS ASADAS MEXICAN GRILL,LAS ASADAS MEXICAN GRILL,2583309.0,Restaurant,Risk 1 (High),3834 W 47TH ST,CHICAGO,IL,60632.0,2019-10-23,Canvass,Out of Business,NaN,41.808025,-87.720037,"{'longitude': '41.80802515275297', 'latitude':..."


### 3- Cleaning of the column *city* and removing the column *state*

*We need to check if there are only information from Chicago. After, we can remove the columns *city* and *state* since we don't need them for further use.*

In [49]:
#check if we have only data from chicage, if not we need to remove all the extra information
data.city.unique()

array(['CHICAGO', nan, 'Chicago', 'CCHICAGO', 'CHICAGO.',
       'CHESTNUT STREET', 'CHICAGOCHICAGO', 'chicago', 'CHICAGOHICAGO',
       'CHicago', '312CHICAGO', 'BEDFORD PARK', 'CHCICAGO',
       'CHARLES A HAYES', 'CHCHICAGO', 'CHICAGOI', 'SUMMIT', 'WESTMONT',
       'LOMBARD', 'INACTIVE', 'BLUE ISLAND'], dtype=object)

> We need to remove :   Bedford Park (Gas Station) ; Blue Island; Lombard ( a village near to Chicago) , Summit ( a city near to Chicago) ; WESTMONT ( village near Chicago) ; aslip (suburb of chicago)
We need to replace by chicago : 'CHARLES A HAYES'  (postal location), 312Chicago (Restaurant) ; CHICAGOI (Chicago)  ; CHESTNUT STREET (street in chicago); INACTIVE (out of business restaurant in Chicago); Chestnut street ;

In [50]:
#Check if the state is unique
display(data.state.unique())

#As the state is unique and we will not use this columns for our further investigations, we can drop it 
data.drop(['state'], axis=1,inplace=True)

#Check if there are other city than Chicago
display(data.inspection_id.groupby(data['city']).count())

#Drop the selected locations 
data = data[~data['city'].isin(["BEDFORD PARK", "BLUE ISLAND", "LOMBARD","SUMMIT","WESTMONT","alsip"])]

#check if the column is clean 
display(data.inspection_id.groupby(data['city']).count())

#now that we are sure that we have only information from Chicago ,we can delete the city columns
data.drop(['city'], axis=1,inplace=True)

array(['IL', nan], dtype=object)

city
312CHICAGO              2
BEDFORD PARK            2
BLUE ISLAND             1
CCHICAGO               44
CHARLES A HAYES         4
CHCHICAGO               6
CHCICAGO                3
CHESTNUT STREET        11
CHICAGO            192693
CHICAGO.                2
CHICAGOCHICAGO          7
CHICAGOHICAGO           2
CHICAGOI                3
CHicago                12
Chicago               317
INACTIVE                8
LOMBARD                 1
SUMMIT                  4
WESTMONT                1
chicago                82
Name: inspection_id, dtype: int64

city
312CHICAGO              2
CCHICAGO               44
CHARLES A HAYES         4
CHCHICAGO               6
CHCICAGO                3
CHESTNUT STREET        11
CHICAGO            192693
CHICAGO.                2
CHICAGOCHICAGO          7
CHICAGOHICAGO           2
CHICAGOI                3
CHicago                12
Chicago               317
INACTIVE                8
chicago                82
Name: inspection_id, dtype: int64

### 4- Cleaning the column *inspection_result*

In [51]:
#looking for the inspection results type
data.groupby('results')['inspection_id'].count()

results
Business Not Located        49
Fail                     37354
No Entry                  6158
Not Ready                 1843
Out of Business          16676
Pass                    104678
Pass w/ Conditions       26566
Name: inspection_id, dtype: int64

> We are only interested in 3 type of results : Pass , Pass with condition and Fail . We need to remove the others.

In [52]:
data = data[~data.results.isin(['Out of Business', 'Business Not Located', 'No Entry','Not Ready'])]

### 5- Cleaning the location information:  *zip* , *latitude* and *longitude*

In [67]:
#make sure that the latitude and longitude are float numbers
data['latitude'] = data['latitude'].astype(float)
data['longitude'] = data['longitude'].astype(float)

#convert the zip into a string (useful for the mapping ) 
data.zip=data.zip.astype(str)

#reformat the zip code writing in order to compare it with the zip code in geojson file (for vizualisation step)
data['zip']=data['zip'].apply(lambda x : x.split('.')[0])

### 6- Cleaning the column *inspection_type*

*We want to focus only on inspection about food and not alcohol. So,we can't keep the 'force task'inspections. Also, we decided to remove the license inspection.*

In [68]:
#Cleaning of the inspections type

searchfor = ['Re', 'Canvass','Food','Complaint']
ignore= ['Fire','Not Ready','Recent Inspection','License','Tag Removal','Recent inspection','SFP']
data=data[data.inspection_type.str.contains('|'.join(searchfor))]
data = data[~data.inspection_type.str.contains('|'.join(ignore))]

data.groupby('inspection_type').count()

,inspection_id,dba_name,aka_name,license,facility_type,risk,address,zip,inspection_date,results,violations,latitude,longitude,location
inspection_type,,,,,,,,,,,,,,
Canvass,81132,81132,80824,81132,80928,81132,81132,81132,81132,81132,77995,81132,81132,81132
Canvass Re-Inspection,20023,20023,19953,20023,20014,20023,20023,20023,20023,20023,14511,20023,20023,20023
Complaint,17178,17178,17106,17178,17158,17178,17178,17178,17178,17178,16802,17178,17178,17178
Complaint Re-Inspection,7346,7346,7314,7346,7341,7346,7346,7346,7346,7346,5246,7346,7346,7346
Short Form Complaint,6565,6565,6535,6565,6559,6565,6565,6565,6565,6565,4848,6565,6565,6565
Suspected Food Poisoning,835,835,832,835,835,835,835,835,835,835,814,835,835,835
Suspected Food Poisoning Re-inspection,191,191,191,191,191,191,191,191,191,191,138,191,191,191


***For a coordination purpose within the group , we will export this clean dataset in a csv file.***

In [69]:
#export the dataframe to a csv 
data.to_csv(r'./clean_dataset.csv')